In [61]:
import config 
import os
import pandas as pd
import ssl
import smtplib

from email.mime.text import MIMEText
from email.utils import make_msgid


email_sender = "hooman.dehghani@squareone.ca"
email_password = config.Gmail_Python_Password

In [74]:
# Read dataframe
directory = r"C:\Users\Hooman Deghani\Python\Data Analysis\Outreach - Skyscraper\Input\Current"
urls = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.csv')]
if len(urls) > 1:
    df = pd.concat([pd.read_csv(url) for url in urls], ignore_index=True)
else: 
    df = pd.read_csv(urls[0])

# Remove columns including "Unnamed"
Unnamed = [column for column in df.columns if "Unnamed" in column]
df = df.drop(columns= Unnamed)

# Add the em[] columns
df.loc[:, ['subject', 'message-id']] = ""

# Temporary: Should be moved to filtering.ipynb
df['Referring Hyperlink'] = ""

In [75]:
# Define the global variables here
context = ssl.create_default_context()
signature = """
<div dir="ltr" data-smartmail="gmail_signature"><div dir="ltr"><span style="color:rgb(34,34,34)">. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .</span><p style="margin:0px"><font color="#f37b20" face="verdana, sans-serif"><span style="font-size:10px"><b>Hooman Dehghani</b></span></font></p><p style="font-size:13px;margin:0px"><span style="font-family:verdana,sans-serif"><span style="color:rgb(127,127,127);font-size:7.5pt">SEO Marketing Coordinator</span></span></p><p style="font-size:13px;margin:0px"><span style="font-family:verdana,sans-serif"><span style="color:rgb(127,127,127);font-size:7.5pt">Square One Insurance Services Inc.</span><span style="font-size:7.5pt"></span></span></p><p style="font-size:13px;margin:0px"><span style="font-family:verdana,sans-serif"><span style="color:rgb(127,127,127);font-size:7.5pt">Suite 1410 -&nbsp;</span><span style="color:rgb(127,127,127);font-size:7.5pt">650&nbsp;West Georgia&nbsp;Street</span></span></p><p style="font-size:13px;margin:0px"><span style="font-family:verdana,sans-serif"><span style="color:rgb(127,127,127);font-size:7.5pt">Vancouver, BC &nbsp;V6B 4N8</span></span></p><p style="font-size:13px;font-family:arial,sans-serif;margin:0px"><span style="color:rgb(80,0,80);font-family:Arial,Helvetica,sans-serif;font-size:small"></span></p><p style="font-size:13px;margin:0px"><span style="font-family:verdana,sans-serif"><span style="color:rgb(127,127,127);font-size:7.5pt">Tel&nbsp;<span style="background-color:rgb(0,0,255)"><span style="color:rgb(0,0,255)"><span style="background-color:rgb(255,255,255)">778.331.6933 ext.207</span></span></span></span></span></p><p style="color:rgb(34,34,34);font-size:13px;margin:0px"><span style="color:rgb(127,127,127);font-size:7.5pt"><font face="verdana, sans-serif"><a value="+17783316933" style="color:rgb(0,0,204)"><span style="color:rgb(127,127,127);font-size:7.5pt">Fax&nbsp;</span></a><a value="+17783316933" style="color:rgb(0,0,204)">1.855.331.6935</a></font></span></p><p style="color:rgb(34,34,34);font-size:13px;font-family:arial,sans-serif;margin:0px"><span style="color:rgb(127,127,127);font-size:7.5pt"><a value="+17783316933" style="color:rgb(0,0,204)"></a></span><span style="font-size:7.5pt"></span></p><p style="color:rgb(34,34,34);font-size:13px;font-family:arial,sans-serif;margin:0px"><span style="color:rgb(127,127,127);font-size:7.5pt"><a value="+17783316933" style="color:rgb(0,0,204)"></a></span></p><p style="color:rgb(34,34,34);font-size:13px;font-family:arial,sans-serif;margin:0px"><span style="color:rgb(109,110,113);font-size:7.5pt">. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .</span></p></div></div>
"""

In [76]:
# Temporary: Fill the empty columns

for i in range(0, 31):
    df.loc[i:, 'Referring Hyperlink'] = f"""<a href="{df.loc[i, "Referring page URL"]}">{df.loc[i, "Referring Topic"]}</a>"""

In [85]:
# Send the emails
for i in range(5, 31):
    email_receiver = df.loc[i, 'Recipient']
    subject = f"Regarding your article on {df.loc[i, 'Referring Topic']}"
    body = f"""
    Hi {df.loc[i, "First Name"]},
<br>
<br>
I noticed that you linked to {df.loc[i, "Target Hyperlink"]} in your article on {df.loc[i, "Referring Hyperlink"]}. I wanted to give you heads up that we’ve created a more in-depth resource that your readers might find useful. It’s like {df.loc[i, 'Target Topic']} but it’s more thorough, up-to-date, and contains peer reviewed information.
<br>
<br>
If you want to use this link in your article instead, here it is: {df.loc[i, "SQ1 URL"]}
<br>
<br>
Either way, thanks for your time.
<br>
<br>
Cheers,
<br>
Hooman
<br>
<br>
{signature}
    """

    ## Send the email
    em = MIMEText(body, "html")
    em['From'] = email_sender
    em['To'] = df.loc[i, "Recipient"]
    em['Subject'] = subject
    em['Message-ID'] = make_msgid()
    em['Thread-Topic'] = subject

    df.loc[i, 'subject'] = subject
    df.loc[i, 'message-id'] = em['Message-ID']

    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as sender:
        sender.login(email_sender, email_password)
        sender.sendmail(email_sender, df.loc[i, "Recipient"], em.as_string())   

 
    

In [89]:
# Save to the same csv file
file_name = f"{df.loc[df.tail(1).index[0], 'Recipient']}.csv"
path = r"C:\Users\Hooman Deghani\Python\Data Analysis\Outreach - Skyscraper\Input\Current\F"
df.to_csv(path+file_name)